In [59]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

### Read transaction data with categories

In [87]:
# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('data/sample/transactions.csv')

# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# remove unnamed column
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,Date,Amount,Description,Category,Subcategory,Type,Year,Month,Month Name
0,01/02/2024,12.44,SPOTIFY 8777781161 NY,Subscription,Music,credit,2024,1,Jan
1,01/02/2024,1487.00,"Zelle payment to PRANAV BHASIN for rent jan""; ...",Financial Services,Payment,debit,2024,1,Jan
2,01/02/2024,49.23,PUBLIX #1131 ORLANDO FL,Merchandise & Supplies,Groceries,credit,2024,1,Jan
3,01/02/2024,-39.00,Zelle payment from ABRAHAM JOSE for Splitwise-...,Financial Services,Transactions,debit,2024,1,Jan
4,01/02/2024,6.40,AplPay PUBLIX SUPERMORLANDO FL,Merchandise & Supplies,Groceries,credit,2024,1,Jan
...,...,...,...,...,...,...,...,...,...
229,12/26/2023,0.98,UBER *TRIP HELP.UBER.C Vorden NLD,Transportation,Taxis & Coach,credit,2023,12,Dec
230,12/26/2023,0.02,FOREIGN TRANSACTION FEE,Fees & Adjustments,Other,credit,2023,12,Dec
231,12/26/2023,25.59,BLAZE PIZZA - WINTER P WINTER PARK FL,Merchandise & Supplies,Groceries,credit,2023,12,Dec
232,12/27/2023,8.49,PUBLIX #1131 ORLANDO FL,Merchandise & Supplies,Groceries,credit,2023,12,Dec


### Data Preprocess

In [88]:
# all rows that have CHEP in description convert category to Salary, and subcategory to W2
df.loc[df['Description'].str.contains('CHEP'), 'Category'] = 'Salary'
df.loc[df['Description'].str.contains('CHEP'), 'Subcategory'] = 'W2'


### Make pie charts - Income/ Expense breakdown

In [98]:
def make_pie_chart_expense_vs_income(df, year, A, B):
    # sub_df = df[(df['Year'] == year)]
    # color_scale = px.colors.qualitative.Set2
    # A vs B

    # count all the positive values as total_income
    total_income = df[(df['Amount'] > 0) & (df['Year'] == year)]['Amount'].sum()
    # count all the negative values as total_expense
    total_expense = df[(df['Amount'] < 0) & (df['Year'] == year)]['Amount'].sum()
    # print(total_income, total_expense)
    if A == 'Income':
        total_text = "$ " + str(round(total_income))
        sub_df = df[(df['Year'] == year) & (df['Amount'] < 0)]
        # convert amount to positive
        sub_df['Amount'] = sub_df['Amount'].abs()
    else:
        total_text = "$ " + str(-round(total_expense))
        sub_df = df[(df['Year'] == year) & (df['Amount'] > 0)]
        # convert amount to positive
        sub_df['Amount'] = sub_df['Amount'].abs()
    # pie of column B vs metric
    color_scale = px.colors.qualitative.Set2
    pie_fig = px.pie(sub_df, values='Amount', names=_type, color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction ='clockwise', hole=0.3, textinfo="label+percent")

    # add total text
    pie_fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',
                        title=dict(text=A+' '+B+" Breakdown "+str(year)),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text, 
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    
    
    return pie_fig, sub_df

In [104]:
# metric = 'Income'
# _type = 'Category'
# income_pie, income_df = make_pie_chart_expense_vs_income(df, 2024, metric, _type)
# income_pie

metric = 'Expense'
_type = 'Category'
expense_pie, expense_df = make_pie_chart_expense_vs_income(df, 2024, metric, _type)
expense_pie

/var/folders/57/qq49wlvd18l75d1d5mcrv4jc0000gn/T/ipykernel_43559/3252291229.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [101]:
# print expense_df.category == 'Payment'
expense_df[expense_df['Subcategory'] == 'Financial Services']

,Date,Amount,Description,Category,Subcategory,Type,Year,Month,Month Name
5,01/03/2024,700.00,ROBINHOOD DES:DEBITS ID:XXXXX9939 INDN:Kesar M...,E-commerce,Financial Services,debit,2024,1,Jan
74,02/14/2024,104.11,Orlando Util Com DES:PAYMENTS ID:XXXXX48816 IN...,Payment,Financial Services,debit,2024,2,Feb
140,03/19/2024,60.00,APPLE CASH SENT M 03/18 PMNT SENT XXXXX38552 CA,Payment,Financial Services,debit,2024,3,Mar
142,03/19/2024,35.00,VENMO DES:PAYMENT ID:XXXXX18584886 INDN:KESAR ...,Payment,Financial Services,debit,2024,3,Mar
149,03/25/2024,2000.00,AMERICAN EXPRESS DES:ACH PMT ID:W5190 INDN:Kes...,Payment,Financial Services,debit,2024,3,Mar
169,04/05/2024,45.00,CASH APP*KEVIN PE 04/05 PMNT SENT XXXXX91940 CA,Payment,Financial Services,debit,2024,4,Apr
201,04/23/2024,1065.00,AMERICAN EXPRESS DES:ACH PMT ID:M6304 INDN:Kes...,Payment,Financial Services,debit,2024,4,Apr


### Make bar charts over months in a year

In [ ]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (EUR)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (EUR)', text_auto='.2s', title=label+" per month", color='Amount (EUR)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

In [ ]:
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
income_monthly_2022

### Putting all charts together into tabs for 2022/2023

In [ ]:
# Pie charts
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
expense_pie_fig_2022 = make_pie_chart(df, 2022, 'Expense')  
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')

# Bar charts
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
expense_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Expense')
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')

# Create tabs
tabs = pn.Tabs(
                        ('2022', pn.Column(pn.Row(income_pie_fig_2022, expense_pie_fig_2022),
                                                pn.Row(income_monthly_2022, expense_monthly_2022))),
                        ('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                pn.Row(income_monthly_2023, expense_monthly_2023))
                        )
                )
tabs.show()

### Create dashboard

In [ ]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"), 
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()